In [50]:
import ollama
import inspect

# Describe tools in JSON schema style
def format_tool(tool_fn):
    sig = inspect.signature(tool_fn)
    params = []
    for name, param in sig.parameters.items():
        params.append({
            "name": name,
            "type": "string",   # keep simple; you can refine if needed
            "description": f"Parameter {name}"
        })
    return {
        "name": tool_fn.__name__,
        "description": tool_fn.__doc__ or "No description",
        "parameters": params
    }

def get_system_tools_prompt(system_prompt: str, tools: list):
    tools_str = "\n".join([str(format_tool(t)) for t in tools])
    return (
        f"{system_prompt}\n\n"
        f"You have access to the following tools. "
        f"When useful, respond ONLY with a JSON object of the form:\n\n"
        f"{{\"name\": <tool_name>, \"parameters\": {{...}}}}\n\n"
        f"Do not answer directly if a tool is needed. Tools:\n{tools_str}"
    )

# Example tool
def get_weather(location: str) -> str:
    """Get weather information for a given location (string)."""
    return f"The weather in {location} is sunny."

def ollama_invoke(user_input: str):
    res = ollama.chat(
        model="llama3-groq-tool-use:8b",
        messages=[
            {
                "role": "system",
                "content": get_system_tools_prompt(
                    system_prompt="You are a helpful assistant that can use tools to answer user questions.",
                    tools=[get_weather]
                )
            },
            {"role": "user", "content": user_input}
        ],
        format="json",
    )
    return res


In [21]:
res.message.content

'{ "I\'m here to assist you with recruitment-related tasks. What do you need help with?": "" }'

In [12]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from typing import Annotated 
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# Agent State Graph

In [40]:

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
graph = StateGraph(AgentState)


In [41]:
graph

# Tools

In [42]:
# static test function
def get_weather(location: str) -> str:
    """Call to get weather information for a given location."""
    return f"The weather in {location} is sunny."

tools = [get_weather]

# Tool Node

In [43]:
tool_node = ToolNode(tools)

graph.add_node("tool_node", tool_node)

# Chat Prompt

In [44]:
def prompt_node(state: AgentState) -> str:
    # send the last message's text (not the whole messages list) to ollama_invoke
    last = state["messages"][-1]
    if isinstance(last, dict) and "content" in last:
        msg_text = last["content"]
    elif hasattr(last, "content"):
        msg_text = last.content
    else:
        msg_text = str(last)

    new_message = ollama_invoke(msg_text)
    return {"messages": [new_message]}
graph.add_node("prompt_node", prompt_node)

# Conditionnal Edge

In [45]:
def conditional_edge(state: AgentState) -> bool:
    last_message = state["messages"][-1]
    print("Last message:", last_message)
    if last_message.tool_calls:
        return "tool_node"  # If a tool was called, go to the ToolNode
    return "__end__"
graph.add_conditional_edges("prompt_node", conditional_edge)

In [46]:
graph.add_edge("tool_node", "prompt_node")
graph.set_entry_point("prompt_node")

# Compile Graph

- error:
- Last message: content='{ "status":"success", "weather": {"temperature":"25°C", "humidity":"60%", "condition":"Partly cloudy"} }' id='c7a7a675-239b-4384-8eb9-86b47e71bd49'

In [51]:
APP= graph.compile()

new_state = APP.invoke({"messages": ['how is the weather today in tunis']})

print(new_state)

NotImplementedError: Unsupported message type: <class 'ollama._types.ChatResponse'>